In [16]:
import os
import random

import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.models import Model

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [17]:
train_folder = "/home/fsantiago/projects/medium/datasets/cifar-10/train"
train_labels = "/home/fsantiago/projects/medium/datasets/cifar-10/trainLabels.csv"

In [18]:
labels_df = pd.read_csv(train_labels, index_col="id")
one_hot = OneHotEncoder().fit(labels_df[["label"]])
one_hot_labels = OneHotEncoder().fit_transform(labels_df[["label"]]).toarray()

In [19]:
def get_y(file):
    file_idx = int(file.replace(".png", "")) - 1

    return one_hot_labels[file_idx]

In [20]:
files = os.listdir(train_folder)
files_batch = files.copy()

In [21]:
train_size = 30000
validation_size = 10000
test_size = 10000

In [22]:
train_files = os.listdir(train_folder)[:train_size]
val_files = os.listdir(train_folder)[train_size:(train_size+validation_size)]
test_files = os.listdir(train_folder)[(train_size+validation_size):(train_size+validation_size+test_size)]

In [23]:
train_X = [plt.imread(f"{train_folder}/{file}") for file in train_files]
train_y = [get_y(file) for file in train_files]

validation_X = [plt.imread(f"{train_folder}/{file}") for file in val_files]
validation_y = [get_y(file) for file in val_files]

test_X = [plt.imread(f"{train_folder}/{file}") for file in test_files]
test_y = [get_y(file) for file in test_files]

train_mean = np.array([train.mean() for train in train_X]).mean()

train_X = [img - train_mean for img in train_X]
validation_X = [img - train_mean for img in validation_X]
test_X = [img - train_mean for img in test_X]

KeyboardInterrupt: 

In [ ]:
batch_size = 1

def load_batches_img_folder(folder_path, train_size, batch_size):

    files = os.listdir(folder_path)[:train_size]
    files_batch = files.copy()
    batches = []

    while len(files_batch) > 0:
        if len(files_batch) < batch_size:
            batch = files_batch.copy()
        else:
            batch = random.sample(files_batch, batch_size)

        files_batch = list(filter(lambda x: x not in batch, files_batch))
        batches.append(batch)
    
    return batches


def load_batches(folder_path, train_size, batch_size, train_mean):
    file_batches = load_batches_img_folder(folder_path, train_size, batch_size)
    batches = []

    for batch_files in file_batches:
        batch = []
        y = []
        for file in batch_files:
            img = plt.imread(f"{folder_path}/{file}") - train_mean
            y.append(get_y(file))
            batch.append(img)

        batches.append((np.array(batch), y))

    return batches

In [ ]:
### Tests purpose only

def test_if_all_files_are_being_used():
    file_batches = load_batches_img_folder(train_folder, batch_size)

    for i in tqdm(range(1, 50001)):

        img_file = f"{i}.png"
        error = True

        for batch in file_batches:
            if img_file in batch:
                error = False

        if error:
            print(img_file)
            break

In [ ]:
batches = load_batches(train_folder, train_size, batch_size, train_mean)

In [ ]:
batches[3][0].mean()

-0.004947779

In [ ]:
auto_input = layers.Input(shape=(32, 32, 3))


def get_conv_layer(filters, x, kernel_size=(3, 3)):

    x = layers.Conv2D(filters, kernel_size=kernel_size, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    return x

def get_conv_layer_2(filters, x, number_of_layers):

    for i in range(number_of_layers - 1):
        x = layers.Conv2D(filters, kernel_size=(3,3), padding="same")(x)

    x = layers.Conv2D(filters, kernel_size=(3,3), strides=(2, 2), padding="same")(x)
    return x


def get_mini_vgg_19():

    x = get_conv_layer(64, auto_input)
    x = get_conv_layer(64, x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = get_conv_layer(128, x)
    x = get_conv_layer(128, x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = get_conv_layer(256, x)
    x = get_conv_layer(256, x)
    x = get_conv_layer(256, x)
    x = get_conv_layer(256, x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = get_conv_layer(512, x)
    x = get_conv_layer(512, x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = layers.Flatten()(x)

    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(250, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(10, activation='softmax')(x)

    return x


def get_mini_plain():

    x = get_conv_layer(64, auto_input, 3)
    x = get_conv_layer(128, x, 4)
    x = get_conv_layer(256, x, 6)
    x = get_conv_layer(512, x, 2)

    x = layers.AveragePooling2D((2, 2), padding="same")(x)

    x = layers.Flatten()(x)

    x = layers.Dense(1000, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(10, activation='softmax')(x)

    return x

x = get_mini_vgg_19()
model = Model(auto_input, x)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       


2022-03-27 13:05:17.995860: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-27 13:05:17.997031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 13:05:17.997274: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 13:05:17.997456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 64)       256       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
          

In [ ]:
optimizer = tf.keras.optimizers.Adam()


def compute_loss(labels, logits):
    loss = tf.keras.losses.categorical_crossentropy(labels, logits)
    return loss


@tf.function
def train_step(x, y): 
    # Use tf.GradientTape()
    with tf.GradientTape() as tape:

        y_hat = model(x)      
        loss = compute_loss(y, y_hat)
       
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
               
        return loss

In [ ]:
epochs = 100
_batches = batches

for epoch in range(epochs):
    for batch in _batches:
        x_batch, y_batch = batch
        x_batch = np.array(x_batch)
        loss = train_step(x_batch, y_batch)

    if epoch % 10 == 0:
        val_predictions = model.predict(np.array(validation_X))
        train_predictions = model.predict(np.array(train_X))

        train_predictions = list(map(lambda x: x[0], one_hot.inverse_transform(train_predictions)))
        train_labels = list(map(lambda x: x[0], one_hot.inverse_transform(train_y)))

        val_predictions = list(map(lambda x: x[0], one_hot.inverse_transform(val_predictions)))
        val_labels = list(map(lambda x: x[0], one_hot.inverse_transform(validation_y)))
        
        train_acc = accuracy_score(train_labels, train_predictions)
        val_acc = accuracy_score(val_labels, val_predictions)

        print(f"{epoch} of {epochs} => Train accuracy: {train_acc} | Validation accuracy: {val_acc}")


2022-03-27 13:05:20.208128: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8300
2022-03-27 13:05:20.564871: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-03-27 13:05:20.889850: W tensorflow/core/common_runtime/bfc_allocator.cc:343] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.


KeyboardInterrupt: 

In [ ]:
batch = 1400
img = 10

val_predictions = model.predict(np.array(validation_X))
test_predictions = model.predict(np.array(test_X))

In [ ]:
val_predictions = list(map(lambda x: x[0], one_hot.inverse_transform(val_predictions)))
val_labels = list(map(lambda x: x[0], one_hot.inverse_transform(validation_y)))

test_predictions = list(map(lambda x: x[0], one_hot.inverse_transform(test_predictions)))
test_labels = list(map(lambda x: x[0], one_hot.inverse_transform(test_y)))

In [ ]:
accuracy_score(test_labels, test_predictions)

0.6923

In [ ]:
accuracy_score(val_labels, val_predictions)

0.6854

In [ ]:
df = pd.DataFrame(train_labels)
df["a"] = df[0]
df.groupby("a").count()

,0
a,
airplane,2997
automobile,2997
bird,2939
cat,2982
deer,3037
dog,2986
frog,3038
horse,2967
ship,3029
